# Babble Tutorial

### You will work with Wikipedia plot descriptions of films that are either comedy or drama.


In [ ]:
from data.preparer import load_film_dataset

## The Data

These plot descriptions are from [Kaggle](https://www.kaggle.com/jrobischon/wikipedia-movie-plots).
You will be labeling films as either "comedy" or "drama" based on their plot descriptions.

Replace PASSWORD with the password to unzip the data, or download it directly from Kaggle.

In [ ]:
!unzip -P PASSWORD data/data.zip
!ls

In [ ]:
df_train, df_dev, df_valid, df_test = load_film_dataset()
print("{} training examples".format(len(df_train)))
print("{} development examples".format(len(df_dev)))
print("{} validation examples".format(len(df_valid)))
print("{} test examples".format(len(df_test)))

In [ ]:
#define labels
ABSTAIN = 0
DRAMA = 1
COMEDY = 2

Transform the data into a format compatible with Babble Labble:

In [ ]:
from babble.Candidate import Candidate # this is a helper class to transform our data into a format Babble can parse

dfs = [df_train, df_dev, df_test]

for df in dfs:
    df["id"] = range(len(df))

Cs = [df.apply(lambda x: Candidate(x), axis=1) for df in dfs]

# babble labble uses 1 and 2 for labels, while our data uses 0 and 1
# add 1 to convert
Ys = [df.label.values + 1 for df in dfs]

In [ ]:
from babble import BabbleStream

aliases = {}
babbler = BabbleStream(Cs, Ys, balanced=True, shuffled=True, seed=456, aliases=aliases)

## Create Explanations

Creating explanations generally happens in five steps:
1. View candidates
2. Write explanations
3. Get feedback
4. Update explanations 
5. Apply label aggregator

Steps 3-5 are optional; explanations may be submitted without any feedback on their quality. However, in our experience, observing how well explanations are being parsed and what their accuracy/coverage on a dev set are (if available) can quickly lead to simple improvements that yield significantly more useful labeling functions. Once a few labeling functions have been collected, you can use the label aggregator to identify candidates that are being mislabeled and write additional explanations targeting those failure modes.

### Collection

Use `babbler` to show candidates

In [ ]:
candidate = babbler.next()
print(candidate)

In [ ]:
explanations = []

In [ ]:
from babble import Explanation
explanation = Explanation(
    name='bounty_hunter', # name of this rule, for your reference
    label=COMEDY, # label to assign
    condition='The phrase "bounty hunter" is in the text', # natural language description of why you label the candidate this way
    candidate=candidate.mention_id # optional argument, the candidate should be an example labeled by this rule
)
explanations.append(explanation)

Babble will parse your explanations into functions, then filter out functions that are duplicates, incorrectly label their given candidate, or assign the same label to all examples.

In [ ]:
parses, filtered = babbler.apply(explanations)

### Analysis
See how your explanations were parsed and filtered

In [ ]:
babbler.analyze(parses)

In [ ]:
babbler.filtered_analysis(filtered)

In [ ]:
babbler.commit()

### Evaluation
Get feedback on the performance of your explanations

In [ ]:
from metal.analysis import lf_summary

Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], Ys[1], lf_names=lf_names)

In [ ]:
from metal import LabelModel
from metal.tuners import RandomSearchTuner

search_space = {
    'n_epochs': [50, 100, 500],
    'lr': {'range': [0.01, 0.001], 'scale': 'log'},
    'show_plots': False,
}

tuner = RandomSearchTuner(LabelModel, seed=123)

label_aggregator = tuner.search(
    search_space, 
    train_args=[Ls[0]], 
    X_dev=Ls[1], Y_dev=Ys[1], 
    max_search=20, verbose=False, metric='f1')

If you'd like to save the explanations you've generated, you can use the `ExplanationIO` object to write to or read them from file.

In [ ]:
from babble.utils import ExplanationIO

FILE = "my_explanations.tsv"
exp_io = ExplanationIO()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)